In [1]:
%matplotlib inline
import numpy as np
from scipy.optimize import minimize
from matplotlib import pyplot as plt

In [2]:
def costfun(n,dt,p,p0):
    d = np.append(np.diff(n), n[0]-n[-1]);
    return dt*np.sum(p*n)+p0*np.sum(p*d*(d>0));

In [3]:
cl1 = (0.22,0.42,0.69)
cl2 = (0.85,0.49,0.19)

In [4]:
T = 24 #number of hours to model
dt = 1
t = np.arange(0,T,dt)

p_n = 1 # maximum number of running nodes

pf = 0.02  # baseline cost ($/CPU hour)
p0 = 0.25 # time wasted in powering on a node (hour/node powered on)
ps = 0.00001 # long term price trend (d($/CPU hour)/d(hour))
  
fs = [0.75, 0.5, 0.25] # percent of maximum load needed 
pvs = [0.01, 0.005, 0.001] # size of daily price fluctuation ($/hour)

In [5]:
ston = []
stop = []
ind = 1
for f in fs:
    for pv in pvs:
        N = f*T*nm # Total amount of node runtime needed
        p = pf + pv*np.sin(2*np.pi*t/24) +ps*t # variable price function
        stop.append(p)
        
        n0 = np.ones(np.size(p)) #initial guess for solution
        
        #Constrain the total usage to be equal to whatever is need
        cons = ({'type': 'eq', 'fun': lambda x:  N - dt*sum(x)})
        
        #Require that no more than the max nodes (nm) are ever running at once
        bnds = tuple((0,nm) for x in n0)
        
        ex = (dt,p,p0)
        opt = {'disp': False, 'iprint': 1, 'eps': 1.4901161193847656e-08, 'maxiter': 100000, 'ftol': 1e-08}
        res = minimize(costfun, n0, args=ex, method='SLSQP', bounds=bnds ,constraints=cons,options=opt)
        print "Success? "+str(res.success)
        ston.append(res.x)
        ind = ind+1


NameError: name 'nm' is not defined

In [ ]:
ind = 1
nN = len(fs);
nP = len(pvs)   
fig = plt.figure(figsize=(10,10))
for f in fs:
    for pv in pvs:
        N = f*T*nm # Total amount of node runtime needed
        p = stop[ind-1] # variable price function
        n = ston[ind-1]
        
        ax1 = plt.subplot(nN,nP,ind)

        if(ind<=nP):
            ax1.set_title("pv = "+str(pv), loc="center")
        
        ax1.plot(t/24., p, color=cl1)
        ax1.set_ylim([0,np.max(pvs)+pf+ps*T])
        ax1.set_xlabel('day')
        if(ind%nP==1):
            ax1.set_title("N/T = "+str(N/T), loc="right",x=-0.3,y=0.6,rotation="vertical")
            # Make the y-axis label and tick labels match the line color.
            ax1.set_ylabel('price', color=cl1)
            for tl in ax1.get_yticklabels():
                tl.set_color(cl1)
        else:
            ax1.get_yaxis().set_ticks([])
        
        ax2 = ax1.twinx()
        ax2.plot(t/24., n, color=cl2)
        ax2.set_ylim([0,550])
            
        if(ind%nP==0):
            ax2.set_ylabel('running nodes', color=cl2)
            for tl in ax2.get_yticklabels():
                tl.set_color(cl2)
        else:
            ax2.get_yaxis().set_ticks([])
        ind = ind+1
plt.suptitle("Optimization of running nodes with different demands (N) and price fluctuations (pv)")